In [2]:
%pip install --upgrade pip
%pip install ipywidgets
%pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio
%pip install --upgrade ipywidgets
%pip install notebook
%pip install jupyterlab
%pip install widgetsnbextension

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [23]:
from datasets import load_dataset, DatasetDict
from huggingface_hub import HfFolder
from datasets import DownloadConfig

# Create a download config with your auth token
download_config = DownloadConfig(token=HfFolder.get_token())

common_voice = DatasetDict()

# Load the dataset with the correct auth configuration
common_voice["train"] = load_dataset(
    "mozilla-foundation/common_voice_11_0",
    "ar",
    split="train+validation",
    trust_remote_code=True,
    download_config=download_config
)

common_voice["test"] = load_dataset(
    "mozilla-foundation/common_voice_11_0",
    "ar",
    split="test",
    trust_remote_code=True,
    download_config=download_config
)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 38481
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10440
    })
})


In [24]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

In [25]:
from transformers import WhisperFeatureExtractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [26]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")

In [27]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 وما أدراك ما يوم الدين
Decoded w/ special:    <|startoftranscript|><|ar|><|transcribe|><|notimestamps|>وما أدراك ما يوم الدين<|endoftext|>
Decoded w/out special: وما أدراك ما يوم الدين
Are equal:             True


In [28]:
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="arabic", task="transcribe")

In [29]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/03e3da23e5fa2d7de56094c40cc7ea529b3b571b215978f053d1fd26557901a6/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([-2.13162821e-14, -1.01252340e-13, -3.55271368e-14, ...,
        1.26173454e-08,  1.54317320e-07,  1.32340631e-07]), 'sampling_rate': 48000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [30]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [31]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/03e3da23e5fa2d7de56094c40cc7ea529b3b571b215978f053d1fd26557901a6/ar_train_0/common_voice_ar_24082672.mp3', 'array': array([ 8.73114914e-11, -4.36557457e-11,  1.67347025e-10, ...,
       -2.33121682e-08, -2.25205440e-07,  4.55183908e-08]), 'sampling_rate': 16000}, 'sentence': 'وما أدراك ما يوم الدين'}


In [32]:
import torch
from transformers import WhisperFeatureExtractor, WhisperTokenizer
from functools import partial

# Force CUDA device
assert torch.cuda.is_available(), "CUDA is required for this script"
device = torch.device("cuda")

# Initialize the feature extractor and tokenizer directly on GPU
feature_extractor = WhisperFeatureExtractor.from_pretrained(
    "openai/whisper-small",
    device_map="cuda"
)
tokenizer = WhisperTokenizer.from_pretrained(
    "openai/whisper-small", 
    language="arabic", 
    task="transcribe",
    device_map="cuda"
)

def prepare_dataset(feature_extractor, tokenizer, batch):
    # Process each audio sample in the batch
    input_features_list = []
    for audio_data in batch["audio"]:
        # compute log-Mel input features from input audio array and move to GPU immediately
        features = feature_extractor(
            audio_data["array"], 
            sampling_rate=audio_data["sampling_rate"],
            return_tensors="pt"
        ).input_features[0].cuda()
        input_features_list.append(features)
    
    # Stack features (already on GPU)
    batch["input_features"] = torch.stack(input_features_list)

    # Tokenize all sentences at once with padding
    encoded = tokenizer(
        batch["sentence"], 
        padding=True,
        return_tensors="pt",
        truncation=True,
        max_length=256  # Adjust this value based on your needs
    ).to(device)
    
    batch["labels"] = encoded.input_ids
    batch["attention_mask"] = encoded.attention_mask
    
    return batch

# Create a partial function with the feature_extractor and tokenizer
prepare_dataset_with_deps = partial(prepare_dataset, feature_extractor, tokenizer)

# Now map the dataset using the partial function with GPU processing
with torch.cuda.device(0):  # Ensure operations happen on GPU
    common_voice = common_voice.map(
        prepare_dataset_with_deps, 
        remove_columns=common_voice.column_names["train"], 
        num_proc=1,  # Using single process for GPU
        batched=True,
        batch_size=32  # Adjust based on your GPU memory
    )

In [33]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [34]:
model.generation_config.language = "arabic"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [35]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [36]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [37]:
import evaluate

metric = evaluate.load("wer")

In [38]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [39]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-arabic",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

/root/MultilingualSummarizerAgenticRAGLLM/final/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/tmp/ipykernel_4101/240849381.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [41]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.110900,0.173932,49.764025
2000,0.099800,0.151588,46.610963
3000,0.068800,0.149143,47.519971
4000,0.062200,0.141933,44.414654
5000,0.038000,0.139593,44.174089


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/root/MultilingualSummarizerAgenticRAGLLM/final/lib/python3.11/site-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553,

TrainOutput(global_step=5000, training_loss=0.09210431472063065, metrics={'train_runtime': 28644.6619, 'train_samples_per_second': 2.793, 'train_steps_per_second': 0.175, 'total_flos': 2.30781744635904e+19, 'train_loss': 0.09210431472063065, 'epoch': 2.0781379883624274})

In [44]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ar, split: test",
    "language": "ar",
    "model_name": "WhisperSmallArabicFineTuned",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [45]:
trainer.push_to_hub(**kwargs)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/humbleakh/whisper-small-arabic/commit/ce58bdc086bbe15d5b7883c3f6ebd7f982e2c5ac', commit_message='End of training', commit_description='', oid='ce58bdc086bbe15d5b7883c3f6ebd7f982e2c5ac', pr_url=None, repo_url=RepoUrl('https://huggingface.co/humbleakh/whisper-small-arabic', endpoint='https://huggingface.co', repo_type='model', repo_id='humbleakh/whisper-small-arabic'), pr_revision=None, pr_num=None)

In [49]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("humbleakh/whisper-small-arabic")
processor = WhisperProcessor.from_pretrained("humbleakh/whisper-small-arabic")


OSError: Can't load tokenizer for 'humbleakh/whisper-small-arabic'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'humbleakh/whisper-small-arabic' is the correct path to a directory containing all relevant files for a WhisperTokenizer tokenizer.

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="humbleakh/WhisperSmallArabicFineTuned")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]  
    return text

iface = gr.Interface(
    fn=transcribe, 
    inputs=gr.Audio(source="microphone", type="filepath"), 
    outputs="text",
    title="Whisper Small Arabic",
    description="Realtime demo for Arabic speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()
